# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
import re
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: Two dictionaries, vocab_to_int, int_to_vocab
    """
    word_counts = Counter(text)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # create int_to_vocab dictionaries
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
        # Replace punctuation with tokens so we can use them in our model
    token_map = {}
    token_map['.'] = '||Period||'
    token_map[','] = '||Comma||'
    token_map['"'] = '||Quotation_Mark||'
    token_map[';'] = '||Semicolon||'
    token_map['!'] = '||Exclamation_Mark||'
    token_map['?'] = '||Question_Mark||'
    token_map['('] = '||Left_Parentheses||'
    token_map[')'] = '||Right_Parentheses||'
    token_map['-'] = '||Dash||'
    token_map['\n']= '||Return||'
    #text = text.replace(':', ' ||COLON|| ')
    #I have added the followed tokens => cuts off 603 redundant words
    token_map['['] = '||Left_Square_Parentheses||'
    token_map[']'] = '||Right_Square_Parentheses||'
    token_map['{'] = '||Left_Curly_Parentheses||'
    token_map['}'] = '||Right_Curly_Parentheses||'
    token_map['*'] = '||Star||'

    # Remove all words with  5 or fewer occurences
    #word_counts = Counter(words)
   # trimmed_words = [word for word in words if word_counts[word] > 5]
    return token_map

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

token_dict : {} {'.': '||Period||', ',': '||Comma||', '"': '||Quotation_Mark||', ';': '||Semicolon||', '!': '||Exclamation_Mark||', '?': '||Question_Mark||', '(': '||Left_Parentheses||', ')': '||Right_Parentheses||', '-': '||Dash||', '\n': '||Return||', '[': '||Left_Square_Parentheses||', ']': '||Right_Square_Parentheses||', '{': '||Left_Curly_Parentheses||', '}': '||Right_Curly_Parentheses||', '*': '||Star||'}
Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)


# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader
import sys

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    batch_size_total = batch_size * sequence_length
    # total number of batches we can make
    n_batches = len(words)//batch_size_total
    # Keep only enough characters to make full batches
    words = words[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    # iterate through the array, one sequence at a time
    x = []
    y = []
    x_data = []
    y_data = []
    for n in range(0, len(words), sequence_length):
        # The features
        if n+sequence_length < len(words):
            x += [words[n:n+sequence_length]]
            # The targets, shifted by one
            y += [words[n+sequence_length]]
    X = torch.tensor(x)
    Y = torch.tensor(y)
    data = TensorDataset(X,Y)
    loader = torch.utils.data.DataLoader(data, batch_size=batch_size,shuffle=True)
    return loader
# there is no test for this function, but you are encouraged to create
# print statements and tests of your own

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [9]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([9, 5])
tensor([[  0,   1,   2,   3,   4],
        [ 30,  31,  32,  33,  34],
        [  5,   6,   7,   8,   9],
        [ 40,  41,  42,  43,  44],
        [ 20,  21,  22,  23,  24],
        [ 35,  36,  37,  38,  39],
        [ 10,  11,  12,  13,  14],
        [ 15,  16,  17,  18,  19],
        [ 25,  26,  27,  28,  29]])

torch.Size([9])
tensor([  5,  35,  10,  45,  25,  40,  15,  20,  30])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [10]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        self.n_layers = n_layers
        print("self.n_layers  {}".format(self.n_layers ))
        self.n_hidden = hidden_dim
        print("self.hidden_dim  {}".format(self.n_hidden ))
        self.embedding_dim = embedding_dim
        print("self.embedding_dim  {}".format(self.embedding_dim ))
        self.vocab_size = vocab_size
        print("self.vocab_size  {}".format(self.vocab_size ))
        self.output_size = output_size
        print("self.output_size  {}".format(self.output_size ))
        self.embed = nn.Embedding(self.vocab_size, self.embedding_dim)
        print("self.embed  {}".format(self.embed ))
         ## TODO: define the layers of the model
        self.lstm = nn.LSTM(self.embedding_dim, self.n_hidden, self.n_layers, 
                            dropout=dropout, batch_first=True)
        print("self.lstm  {}".format(self.embed ))
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(dropout)
        print("self.dropout  {}".format(self.embed ))
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(self.n_hidden, self.vocab_size)
        print("self.fc  {}".format(self.embed ))
        # TODO: Implement function
    
    def print_enable(self,msg, enable):
        if (enable): #global variable
            print(msg)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function
       # nn_input = torch.IntTensor(nn_input)
        out = self.embed(nn_input) 
        out, hidden = self.lstm(out, hidden)

        ## TODO: pass through a dropout layer
        out = self.dropout(out)

        out = out.contiguous().view(-1, self.n_hidden)

        # return one batch of output word scores and the hidden stat
        ## TODO: put x through the fully-connected layer
        
        out = self.fc(out)
        batch_size = nn_input.shape[0]
        out = out.view(batch_size, -1, self.output_size)
        # get last batch

        out = out[:, -1]

        # return the final output and the hidden state
        return out, hidden
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
         
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)


self.n_layers  2
self.hidden_dim  10
self.embedding_dim  15
self.vocab_size  20
self.output_size  20
self.embed  Embedding(20, 15)
self.lstm  Embedding(20, 15)
self.dropout  Embedding(20, 15)
self.fc  Embedding(20, 15)
Tests Passed


In [12]:
class AssertTest(object):
    def __init__(self, params):
        self.assert_param_message = '\n'.join([str(k) + ': ' + str(v) + '' for k, v in params.items()])
    
    def test(self, assert_condition, assert_message):
        assert assert_condition, assert_message + '\n\nUnit Test Function Parameters\n' + self.assert_param_message

batch_size = 50
sequence_length = 3
vocab_size = 20
output_size=20
embedding_dim=15
hidden_dim = 10
n_layers = 2

# create test RNN
# params: (vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

# create test input
a = np.random.randint(vocab_size, size=(batch_size, sequence_length))
#b = torch.LongTensor(a)
b = torch.from_numpy(a)
print("input : {}".format(b.shape))
hidden = rnn.init_hidden(batch_size)
print("hidden : {}".format(len(hidden)))

if(train_on_gpu):
    rnn.cuda()
    b = b.cuda()

output, hidden_out = rnn.forward(b, hidden)

assert_test = AssertTest({
                         'Input Size': vocab_size,
                         'Output Size': output_size,
                         'Hidden Dim': hidden_dim,
                         'N Layers': n_layers,
                         'Batch Size': batch_size,
                         'Sequence Length': sequence_length,
                         'Input': b})

# initialization
correct_hidden_size = (n_layers, batch_size, hidden_dim)
assert_condition = hidden[0].size() == correct_hidden_size
assert_message = 'Wrong hidden state size. Expected type {}. Got type {}'.format(correct_hidden_size, hidden[0].size())
assert_test.test(assert_condition, assert_message)

# output of rnn
correct_hidden_size = (n_layers, batch_size, hidden_dim)
assert_condition = hidden_out[0].size() == correct_hidden_size
assert_message = 'Wrong hidden state size. Expected type {}. Got type {}'.format(correct_hidden_size, hidden_out[0].size())
assert_test.test(assert_condition, assert_message)

correct_output_size = (batch_size, output_size)
assert_condition = output.size() == correct_output_size
assert_message = 'Wrong output size. Expected type {}. Got type {}'.format(correct_output_size, output.size())
assert_test.test(assert_condition, assert_message)

#_print_success_message()

self.n_layers  2
self.hidden_dim  10
self.embedding_dim  15
self.vocab_size  20
self.output_size  20
self.embed  Embedding(20, 15)
self.lstm  Embedding(20, 15)
self.dropout  Embedding(20, 15)
self.fc  Embedding(20, 15)
input : torch.Size([50, 3])
hidden : 2


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [13]:
import sys
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((*arr.size(), n_labels), dtype=np.long)
    one_hot[np.arange(arr.size()[0]), arr] = 1.
    one_hot = one_hot.reshape((*arr.size(), n_labels))
    return (torch.from_numpy(one_hot)).type(torch.LongTensor)

def inspectInput(tensor):
    tensor_shape = tensor.shape
    print(tensor_shape)
    outer_1 = []
    for i in range(tensor_shape[0]):
        outer_2 = []
        if len(tensor_shape) > 1:
            for j in range(tensor_shape[1]):
                if len(tensor_shape) > 2:
                    outer_3 = []
                    for k in range(tensor_shape[2]):
                        outer_3 += [int_to_vocab[np.asscalar(tensor[i][j][k].numpy().astype(int))]]
                    outer_2 +=[outer_3]
                elif len(tensor_shape) == 2:
                    outer_2 +=[int_to_vocab[np.asscalar(tensor[i][j].numpy().astype(int))]]
            outer_1+=[outer_2]
        elif len(tensor_shape) == 1:
            outer_1+=[int_to_vocab[np.asscalar(tensor[i].numpy().astype(int))]]
    return outer_1
        
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
  #  print("input : {}".format(inp))
    # move data to GPU, if available
    # I have had to change the tensor from Float to Long, how to better generalise?
    inputs, targets = torch.LongTensor(inp), torch.LongTensor(target)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    inputs, targets = inputs.to(device), targets.to(device)
    out, hidden = rnn.forward(inputs, hidden)
    loss = criterion(out, targets)
    #rolling average
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

"\nDON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE\n"

## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import sys
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, save_name, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        print("epoch_i : {}".format(epoch_i))
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
     #   print("init hidden : ({},{})".format(hidden[0].shape, hidden[1].shape))
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
         #   print(batch_i)
          #  print(inputs, labels)
            #sys.exit(0)
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden) 
           # print("batch_i : {} loss : {}".format(batch_i,loss))
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Batch : {}/{} Epoch: {:>4}/{:<4}  Loss: {}\n'.format(batch_i, n_batches,
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []
                helper.save_model('./save/{}'.format(save_name), rnn)

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [15]:
# Data params
# Sequence Length
sequence_length = 3 # of words in a sequence
# Batch Size
batch_size =256
print("int_text : {}".format(int_text))
# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
# Training parameters
# Number of Epochs
num_epochs = 30
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 10
# Hidden Dimension
hidden_dim = 256
#hidden_dim = len(vocab_to_int)
# Number of RNN Layers
n_layers = 1

# Show stats for every n number of batches
show_every_n_batches = 50

save_name = "trained_rnn1"

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
#trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, save_name, show_every_n_batches)

# saving the trained model
#helper.save_model('./save/{}'.format(save_name), trained_rnn)
print('Model Trained and Saved')

self.n_layers  1
self.hidden_dim  256
self.embedding_dim  10
self.vocab_size  20785
self.output_size  20785
self.embed  Embedding(20785, 10)
self.lstm  Embedding(20785, 10)
self.dropout  Embedding(20785, 10)
self.fc  Embedding(20785, 10)
Training for 30 epoch(s)...
epoch_i : 1


/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Batch : 50/1163 Epoch:    1/30    Loss: 7.277223119735718



/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Batch : 100/1163 Epoch:    1/30    Loss: 5.905791816711425

Batch : 150/1163 Epoch:    1/30    Loss: 5.8271488857269285

Batch : 200/1163 Epoch:    1/30    Loss: 5.757176609039306

Batch : 250/1163 Epoch:    1/30    Loss: 5.693898115158081

Batch : 300/1163 Epoch:    1/30    Loss: 5.639876708984375

Batch : 350/1163 Epoch:    1/30    Loss: 5.569299488067627

Batch : 400/1163 Epoch:    1/30    Loss: 5.487158432006836

Batch : 450/1163 Epoch:    1/30    Loss: 5.4196383571624756

Batch : 500/1163 Epoch:    1/30    Loss: 5.387648506164551

Batch : 550/1163 Epoch:    1/30    Loss: 5.3035165882110595

Batch : 600/1163 Epoch:    1/30    Loss: 5.387282228469848

Batch : 650/1163 Epoch:    1/30    Loss: 5.371204996109009

Batch : 700/1163 Epoch:    1/30    Loss: 5.307073221206665

Batch : 750/1163 Epoch:    1/30    Loss: 5.350691261291504

Batch : 800/1163 Epoch:    1/30    Loss: 5.290316839218139

Batch : 850/1163 Epoch:    1/30    Loss: 5.2108360767364506

Batch : 900/1163 Epoch:    1/30    L

Batch : 1150/1163 Epoch:    6/30    Loss: 4.326240653991699

epoch_i : 7
Batch : 50/1163 Epoch:    7/30    Loss: 4.236924894272335

Batch : 100/1163 Epoch:    7/30    Loss: 4.250049343109131

Batch : 150/1163 Epoch:    7/30    Loss: 4.242657432556152

Batch : 200/1163 Epoch:    7/30    Loss: 4.219705080986023

Batch : 250/1163 Epoch:    7/30    Loss: 4.208672728538513

Batch : 300/1163 Epoch:    7/30    Loss: 4.2773174285888675

Batch : 350/1163 Epoch:    7/30    Loss: 4.2101916360855105

Batch : 400/1163 Epoch:    7/30    Loss: 4.259025273323059

Batch : 450/1163 Epoch:    7/30    Loss: 4.307139234542847

Batch : 500/1163 Epoch:    7/30    Loss: 4.250778870582581

Batch : 550/1163 Epoch:    7/30    Loss: 4.258832311630249

Batch : 600/1163 Epoch:    7/30    Loss: 4.316668410301208

Batch : 650/1163 Epoch:    7/30    Loss: 4.253321223258972

Batch : 700/1163 Epoch:    7/30    Loss: 4.269561748504639

Batch : 750/1163 Epoch:    7/30    Loss: 4.258242845535278

Batch : 800/1163 Epoch:   

Batch : 1000/1163 Epoch:   12/30    Loss: 3.939404983520508

Batch : 1050/1163 Epoch:   12/30    Loss: 3.9735638761520384

Batch : 1100/1163 Epoch:   12/30    Loss: 3.926181516647339

Batch : 1150/1163 Epoch:   12/30    Loss: 3.973531470298767

epoch_i : 13
Batch : 50/1163 Epoch:   13/30    Loss: 3.9125275157746815

Batch : 100/1163 Epoch:   13/30    Loss: 3.8624450635910033

Batch : 150/1163 Epoch:   13/30    Loss: 3.867962875366211

Batch : 200/1163 Epoch:   13/30    Loss: 3.881061496734619

Batch : 250/1163 Epoch:   13/30    Loss: 3.879627504348755

Batch : 300/1163 Epoch:   13/30    Loss: 3.8926128196716308

Batch : 350/1163 Epoch:   13/30    Loss: 3.876728973388672

Batch : 400/1163 Epoch:   13/30    Loss: 3.857350301742554

Batch : 450/1163 Epoch:   13/30    Loss: 3.927472543716431

Batch : 500/1163 Epoch:   13/30    Loss: 3.9108535957336428

Batch : 550/1163 Epoch:   13/30    Loss: 3.8539918756484983

Batch : 600/1163 Epoch:   13/30    Loss: 3.8881802320480348

Batch : 650/1163 

Batch : 800/1163 Epoch:   18/30    Loss: 3.6938412475585936

Batch : 850/1163 Epoch:   18/30    Loss: 3.6689901876449587

Batch : 900/1163 Epoch:   18/30    Loss: 3.730845217704773

Batch : 950/1163 Epoch:   18/30    Loss: 3.779909372329712

Batch : 1000/1163 Epoch:   18/30    Loss: 3.7402802753448485

Batch : 1050/1163 Epoch:   18/30    Loss: 3.7440318489074706

Batch : 1100/1163 Epoch:   18/30    Loss: 3.7546753883361816

Batch : 1150/1163 Epoch:   18/30    Loss: 3.7362269020080565

epoch_i : 19
Batch : 50/1163 Epoch:   19/30    Loss: 3.65799291550167

Batch : 100/1163 Epoch:   19/30    Loss: 3.673373804092407

Batch : 150/1163 Epoch:   19/30    Loss: 3.665453119277954

Batch : 200/1163 Epoch:   19/30    Loss: 3.6294678401947023

Batch : 250/1163 Epoch:   19/30    Loss: 3.690918836593628

Batch : 300/1163 Epoch:   19/30    Loss: 3.640614824295044

Batch : 350/1163 Epoch:   19/30    Loss: 3.6645571517944338

Batch : 400/1163 Epoch:   19/30    Loss: 3.6573685693740843

Batch : 450/1163

Batch : 600/1163 Epoch:   24/30    Loss: 3.557392029762268

Batch : 650/1163 Epoch:   24/30    Loss: 3.5417986631393434

Batch : 700/1163 Epoch:   24/30    Loss: 3.604833288192749

Batch : 750/1163 Epoch:   24/30    Loss: 3.587783570289612

Batch : 800/1163 Epoch:   24/30    Loss: 3.5563403749465943

Batch : 850/1163 Epoch:   24/30    Loss: 3.6128097915649415

Batch : 900/1163 Epoch:   24/30    Loss: 3.5916270446777343

Batch : 950/1163 Epoch:   24/30    Loss: 3.5989414119720458

Batch : 1000/1163 Epoch:   24/30    Loss: 3.5752149629592895

Batch : 1050/1163 Epoch:   24/30    Loss: 3.6039372634887696

Batch : 1100/1163 Epoch:   24/30    Loss: 3.60942186832428

Batch : 1150/1163 Epoch:   24/30    Loss: 3.6132195806503296

epoch_i : 25
Batch : 50/1163 Epoch:   25/30    Loss: 3.4939043484036882

Batch : 100/1163 Epoch:   25/30    Loss: 3.526472134590149

Batch : 150/1163 Epoch:   25/30    Loss: 3.491792287826538

Batch : 200/1163 Epoch:   25/30    Loss: 3.462507824897766

Batch : 250/1163

Batch : 400/1163 Epoch:   30/30    Loss: 3.4413089656829836

Batch : 450/1163 Epoch:   30/30    Loss: 3.420242261886597

Batch : 500/1163 Epoch:   30/30    Loss: 3.4499411725997926

Batch : 550/1163 Epoch:   30/30    Loss: 3.457538061141968

Batch : 600/1163 Epoch:   30/30    Loss: 3.442528553009033

Batch : 650/1163 Epoch:   30/30    Loss: 3.4703566646575927

Batch : 700/1163 Epoch:   30/30    Loss: 3.473219118118286

Batch : 750/1163 Epoch:   30/30    Loss: 3.4752196359634397

Batch : 800/1163 Epoch:   30/30    Loss: 3.471260013580322

Batch : 850/1163 Epoch:   30/30    Loss: 3.4807781171798706

Batch : 900/1163 Epoch:   30/30    Loss: 3.4811371994018554

Batch : 950/1163 Epoch:   30/30    Loss: 3.4793490076065066

Batch : 1000/1163 Epoch:   30/30    Loss: 3.504822006225586

Batch : 1050/1163 Epoch:   30/30    Loss: 3.5171544790267943

Batch : 1100/1163 Epoch:   30/30    Loss: 3.526127805709839

Batch : 1150/1163 Epoch:   30/30    Loss: 3.4725466442108153

Model Trained and Saved


In [ ]:
#continuation of training

#rnn = helper.load_model('./save/{}'.format(save_name))
print(trained_rnn)
optimizer = torch.optim.Adam(trained_rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
show_every_n_batches = 50
num_epochs = 5
# training the model
#trained_rnn = train_rnn(trained_rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** 
The key hyperparameter to ensure greater learning is the batch_size I found that increasing batch size from 50 to 256 crucially sped up the training. The large overhead of training per batch is spread over more sequences. I tried increasing hidden_dim to larger  values from 64 to 256, settling on the larger. I kept the FC layer at one hidden layer. After studying the script I found many short sentences and attempted a shorter SEQ length =3. I would like to try SEQ length=5 but due to GPU-time-cost of running (12hours!) I will have to leave this for now.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
#trained_rnn = rnn
trained_rnn = helper.load_model('./save/{}'.format(save_name))


## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    it = 0
    for _ in range(predict_len):
        it+=1
      #  print("it : {}".format(it))
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        # get the output of the rnn
        output, _ = rnn.forward(current_seq, hidden)
     #   print("output : {}".format(output))
    #    # get the next word probabilities
        p = F.softmax(output, dim=1).data
      #  print("p : {}".format(p))
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
     #   print(p)
       # print(top_i)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i

    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [26]:
# run the cell multiple times to get different results!
gen_length = 1000 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
#print("pad_word {}".format(pad_word))
#print("vocab_to_int[pad_word] {}".format(vocab_to_int[pad_word]))
#print(int_to_vocab)
#l = list(vocab_to_int.keys())
#l.sort()
#print(l)
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry:?

george: yeah, yeah, yeah, yeah.

kramer: well, i think i'm going to be in the park, it's not.

george: i don't know.(jerry nods)

george: i can't believe it.

elaine: well, i got a date?

jerry:(pause) oh!

elaine: oh, i don't know what i was just the only of time. i don't know, you know, i think i can get you to do it!

elaine:(to the phone)

kramer: well, i don't want to talk to him.

george: well, i can't have a great time.

elaine:(to george) you don't think we should be the executor be the only one.

kramer: i think she would be like you?

jerry: oh, you are not?

jerry: yeah.

george: i think it's a good thing.

elaine:(to kramer)

george:(shouting) yeah.(he hangs up)

jerry: no, i don't know what?

george: i don't know. i can't do this, i don't think it was.

george: i think i should do?

jerry:(pleading) oh, no!

kramer: yeah, i think you should have to say, i think i could have to be here.(to george)

elaine:(gasps) oh, i don't know how to be in the park?

newman:(sho

In [ ]:
int_to_vocab[3]

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [25]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.